## Oblivious Transfer Algorithm

### Motivation

Let's imagine that we have two people: Paula and Valerie.

Paula knows a function $f: bool \rightarrow int$.

Valerie wants to query Paula for $f(true)$. The easiest way is for Valerie to simply ask for the value.

In [2]:
class Paula:
    def __init__(self):
        self.friend = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.friend = None
        self.queried_value = True

    def get_f(self):
        value = self.friend.send_f(self.queried_value)
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.friend = self.valerie
        self.valerie.friend = self.paula

protocol = Protocol()
value = protocol.valerie.get_f()

Paula recieves parameter True.
Paula sends value 1000.
Valerie obtains value 1000.


Let's imagine now that Valerie doesn't want to tell Paula that she wants the value of $f$ for parameter $True$. She wants this transfer to be oblivious. The easiest way to do this is to ask for both of them, so Paula wouldn't know which value Valerie really wants.

In [3]:
class Paula:
    def __init__(self):
        self.friend = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.friend = None
        self.queried_value = True

    def get_f(self):
        value_for_true = self.paula.send_f(True)
        value_for_false = self.paula.send_f(False)
        
        value = value_for_true if self.queried_value else value_for_false
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula
        
protocol = Protocol()
value = protocol.valerie.get_f()

Paula recieves parameter True.
Paula sends value 1000.
Paula recieves parameter False.
Paula sends value 500.
Valerie obtains value 1000.


This works okay for Valerie, whose privacy is respected, but what if Paula doesn't want to give too much information either?

Notice that in the first naive version Paula only gives one result to Valerie but in the second version she gives her both. This is not desirable.

We'll see a protocol that allows both parties to mantain privacy. That is:
- Paula only reveals one value to Valerie
- Valerie doesn't reveal which of the values she wants

We're going to need that both parties share a RSA key pair to be able to hide the requests.

The algorithm is as follows:

##### Idea
- Paula offers Valerie two random values, $x_{false}$ for $f(false)$ and $x_{true}$ for $f(true)$.
- Valerie chooses the $x_b$ that corresponds to the bit $b$ she wants to know. She then generates her own random k and uses it to hide $x_b$ in a number $k'$.
- Paula receives the value $k'$ and does 2 calculations: one assuming that Valerie chose $x_{false}$ and the other one assuming she chose $x_{true}$. One of them makes sense and the other one doesn't, but Paula doesn't know which is which.
- So Paula sends both results. Valerie knows which value makes sense and which doesn't, so she obtains the result with one and discards the other.

##### Diagram

| Paula                                                                            |   Communication    | Valerie                                     |
| -------------------------------------------------------------------------------- |:-------------:| ----------------------------------------- |
|                                                                                  |               | Wants to know $f(b)$ and then asks for the $x_b$s                 |
| Generates random values $x_{false}, x_{true}$                                                        | $\rightarrow$ | $x_{false}, x_{true}$                                |
|                                                                                  |               | Generates random noise $k$                        |
| $k'$                                                                              | $\leftarrow$  | Calculates $k' := (x_b + k^e) \text{ mod } N$ |
| Calculates: $k_{false} := (k' - x_{false})^d \text{ mod } N$ y $k_{true} := (k' - x_{true})^d \text{ mod } N$ |            |                                   ($k_b = k$, and $k_{!b}$ is noise)      |
| Calculates $v'_{false} := (v_{false} + k_{false}) \text{ mod } N$ y $v'_{true} := (v_{true} + k_{true}) \text{ mod } N$ |       $\rightarrow$        |                                      $v'_0, v'_1$     |
| | | Obtains $v_b = v'_b - k$ and discards $v_{!b}$ |

In [8]:
from Crypto.PublicKey import RSA
import random

def random_noise():
    # N is a really big number
    N = 2**2048
    return random.randrange(N)

class Paula:
    def __init__(self, key, modulus):
        self.friend = None
        
        self.key = key
        self.modulus = modulus
        self.x_bs = {
            True: random_noise(),
            False: random_noise(),
        }
    
    def get_x_bs(self):
        print(f"\nPaula sends x_bs {self.x_bs}.")
        return self.x_bs
    
    def calculate_k_b(self, bit, encrypted_key):
        #Complete
        
    def calculate_v_b(self, k_b, bit):
        #Complete
    
    def calculate_encrypted_value(self, bit, encrypted_key):
        k_b = self.calculate_k_b(bit, encrypted_key)
        return self.calculate_v_b(k_b, bit)
    
    def get_f(self, encrypted_key):
        print(f"\nPaula recieves parameter {encrypted_key}.")
    
        values = {
            b: self.calculate_encrypted_value(b, encrypted_key) 
            for b in [True, False]
        }
        print(f"\nPaula sends values {values}.")
        return values
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self, key, modulus):
        self.friend = None
        self.queried_key = True
        self.key = key
        self.modulus = modulus
        
        self.k = random_noise()

    def get_correct_xb(self):
        #Complete
    
    def get_correct_encrypted_value(self, hidden_key):
        #Complete
    
    def hide_key(self, xb):
        #Complete
    
    def decrypt_value(self, encrypted_value):
        #Complete
    
    def get_f(self):
        xb = self.get_correct_xb()
        print(f"\nValerie chooses correct xb {xb}.")
        
        hidden_key = self.hide_key(xb)
        print(f"\nValerie calculates hidden key {hidden_key}.")
        
        encrypted_value = self.get_correct_encrypted_value(hidden_key)
        print(f"\nValerie receives hidden value {encrypted_value}.")
        
        value = self.decrypt_value(encrypted_value)
        print(f"\nValerie decrypts value {value}.")
    
class Protocol:
    def __init__(self):
        key_pair = RSA.generate(bits=2048)
        paula_key = key_pair.d
        valerie_key = key_pair.e
        modulus = key_pair.n
        
        self.paula = Paula(paula_key, modulus)
        self.valerie = Valerie(valerie_key, modulus)
        
        self.paula.friend = self.valerie
        self.valerie.friend = self.paula
        
protocol = Protocol()
value = protocol.valerie.get_f()


Paula sends x_bs {True: 29958692151612506224195786899995064429604672355953062214948850119211868083797431944225081807377968639652629361797931393973199343016580423747768343936144066710192100724173312120696397183918532141400301794621027481982387056049393875551218122030122067530800730707507944427882270177990470650051494891156796013870081452381122807099261186548991435512583094021285437307034234478107514880651826929137222047932720501553353902149689145255117449023065547196227207592766832280982381560091768191461769479142691768517318823710655064312905582466083709684693038290562432212266619106639855418273931356384239787180534051410869446794169, False: 2445296497957913885593448793147281630791649597409239721010208578416421674315598307010981932006396450118015543593670911428997880959229120471598959258149569592216380352124795887547467092099136051811880310308338957128240712378965693823292495623479827143413067247779467197383304837242470358659878132686486517338292349957331078133770537434431139605856620

##### Explanation
Here we'll explain why this works.

Valerie calculates $k'$ as:

$k' := (x_b + k^e) \text{ mod } N$

When Paula calculates $k_b$, the following happens (we'll omit the modulo $N$ notation):

$k_b = (k' - x_b)^d = (x_b + k^e - x_b)^d = (k^e)^d = k$

This is true because $e$ and $d$ are a pair of RSA keys. On the other hand, for $k_{!b}$:

$k_{!b} = (k' - x_{!b})^d = (x_b + k^e - x_{!b})^d$

Which is a random value.

For Paula both values are random because she never saw $k$, only $k'$, so she can't recognise it.

This means that Paula sends to Valerie:
- $v'_b = v_b - k$
- $v'_{!b} = v_{!b} - (x_b + k^e - x_{!b})^d$

Valerie knows $k$ so she can obtain $v_b$. She can't calculate $(x_b + k^e - x_{!b})^d$ because she doesn't know $d$, so she can't decypher $v_{!b}$.